In [1]:
# 自动计算cell的计算时间
%load_ext autotime

%matplotlib inline
%config InlineBackend.figure_format='svg' #矢量图设置，让绘图更清晰

time: 551 ms (started: 2021-08-29 13:31:15 +08:00)


In [2]:
%%bash

# 增加更新
git add *.ipynb *.md

git remote -v

git commit -m '更新 #2 Aug 29, 2021'

#git push origin master
git push

origin	git@github.com:ustchope/pytorch_lightning_study.git (fetch)
origin	git@github.com:ustchope/pytorch_lightning_study.git (push)
[main 88ceda3] 更新 #2 Aug 29, 2021
 2 files changed, 2780 insertions(+), 408 deletions(-)
 create mode 100644 Trainer.ipynb


To git@github.com:ustchope/pytorch_lightning_study.git
   988c748..88ceda3  main -> main


In [2]:
import os
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, random_split
import pytorch_lightning as pl

time: 3.75 s (started: 2021-08-29 13:31:16 +08:00)


将 PyTorch 代码组织到 LightningModule 中后，Trainer 会自动执行其他所有操作。

这种抽象实现了以下目标：
1. 您可以通过 PyTorch 代码保持对所有方面的控制，而无需添加抽象。
2. 培训师使用来自 Facebook AI Research、纽约大学、麻省理工学院、斯坦福大学等顶级 AI 实验室的贡献者和用户嵌入的最佳实践......
3. 培训师允许覆盖您不希望自动化的任何关键部分。

# 基本使用

这是训练器的基本用法：

In [ ]:
model = MyLightningModule()

trainer = Trainer()
trainer.fit(model, train_dataloader, val_dataloader)

# 引擎盖下

在幕后，Lightning Trainer 会为您处理训练循环的详细信息，一些示例包括：
* 自动启用/禁用梯度
* 运行训练、验证和测试数据加载器
* 在适当的时间调用回调
* 将批处理和计算放在正确的设备上

这是训练器在幕后所做的伪代码（仅显示训练循环）

In [ ]:
# put model in train mode
model.train()
torch.set_grad_enabled(True)

losses = []
for batch in train_dataloader:
    # calls hooks like this one
    on_train_batch_start()

    # train step
    loss = training_step(batch)

    # clear gradients
    optimizer.zero_grad()

    # backward
    loss.backward()

    # update parameters
    optimizer.step()

    losses.append(loss)

# Python 脚本中的培训师

在 Python 脚本中，建议您使用 main 函数来调用 Trainer。

In [ ]:
from argparse import ArgumentParser


def main(hparams):
    model = LightningModule()
    trainer = Trainer(gpus=hparams.gpus)
    trainer.fit(model)


if __name__ == "__main__":
    parser = ArgumentParser()
    parser.add_argument("--gpus", default=None)
    args = parser.parse_args()

    main(args)

所以你可以像这样运行它：

In [ ]:
python main.py --gpus 2

> 专业提示：您不需要手动定义所有标志。 PL可以自动添加它们

In [ ]:
from argparse import ArgumentParser


def main(args):
    model = LightningModule()
    trainer = Trainer.from_argparse_args(args)
    trainer.fit(model)


if __name__ == "__main__":
    parser = ArgumentParser()
    parser = Trainer.add_argparse_args(parser)
    args = parser.parse_args()

    main(args)

所以你可以像这样运行它：

In [ ]:
python main.py --gpus 2 --max_steps 10 --limit_train_batches 10 --any_trainer_arg x

> 如果您想提前停止训练运行，可以按键盘上的“Ctrl + C”。 训练器将捕获 KeyboardInterrupt 并尝试正常关闭，包括运行加速器回调 on_train_end 以清理内存。 在这种情况下，训练器对象还会将中断的属性设置为 True。 例如，如果您有关闭计算资源的回调，您可以有条件地运行关闭逻辑，仅用于不间断运行。

# 验证

您可以在训练循环之外使用 pytorch_lightning.trainer.trainer.Trainer.validate() 对验证集执行评估时期。 如果您想在模型初始化时或已经训练后从模型中收集新指标，这可能很有用。

In [ ]:
trainer.validate(dataloaders=val_dataloaders)

# 测试

完成训练后，请随时运行测试集！ （仅在发表论文或投入生产之前）

In [ ]:
trainer.test(test_dataloaders=test_dataloaders)

# 再现性

为了确保从运行到运行的完全可重复性，您需要为伪随机生成器设置种子，并在 Trainer 中设置确定性标志。

例子：

In [ ]:
from pytorch_lightning import Trainer, seed_everything

seed_everything(42, workers=True)
# 为 numpy、torch、python.random 和 PYTHONHASHSEED 设置种子。
model = Model()
trainer = Trainer(deterministic=True)

通过在`seed_everything()` 中设置`workers=True`，Lightning 为torch、numpy 和stdlib 随机数生成器的所有数据加载器worker 和进程派生出唯一的种子。 当打开时，它确保例如 数据增强不会在工作人员之间重复。

# Trainer flags

## accelerator

要使用的加速器后端（以前称为分布式后端）。
* ('dp') 是 DataParallel（在同一台机器的 GPU 之间拆分批次）
* ('ddp') 是 DistributedDataParallel（每个节点上的每个 gpu 训练，并同步 grads）
* ('ddp_cpu') 是 CPU 上的 DistributedDataParallel（与 'ddp' 相同，但不使用 GPU。用于多节点 CPU 训练或单节点调试。请注意，这不会在单节点上提供加速，因为 Torch 已经在一台机器上有效地利用了多个 CPU。）
* ('ddp2') 节点上的 dp，节点间的 ddp。 对增加负样本数量等有用

In [ ]:
# 培训师使用的默认值
trainer = Trainer(accelerator=None)

例子：

In [ ]:
# dp = DataParallel
trainer = Trainer(gpus=2, accelerator='dp')

# ddp = DistributedDataParallel
trainer = Trainer(gpus=2, num_nodes=2, accelerator='ddp')

# ddp2 = DistributedDataParallel + dp
trainer = Trainer(gpus=2, num_nodes=2, accelerator='ddp2')

> 此选项不适用于 TPU。 TPU 默认使用“ddp”（在每个核心上）

您还可以通过对现有加速器进行子类化来修改硬件行为以适应您的需要。

例子：

In [ ]:
class MyOwnAcc(Accelerator):
    ...

Trainer(accelerator=MyOwnAcc())

> 传入自定义加速器是实验性的，但正在进行工作以实现完全兼容性。

## accumulate_grad_batches

每 k 个批次或按照 dict 中的设置累积 grads。 Trainer 还为最后一个不可分割的步数调用 optimizer.step()。

In [ ]:
# default used by the Trainer (no accumulation)
trainer = Trainer(accumulate_grad_batches=1)

例子：

In [ ]:
# 每 4 个批次累积一次（有效批次大小为批次*4）
trainer = Trainer(accumulate_grad_batches=4)

# 第 1-4 轮没有积累。 在第 5-10 个时期累积 3。 之后累积 20
trainer = Trainer(accumulate_grad_batches={5: 3, 10: 20})

## amp_backend

使用 PyTorch AMP（‘native’）（可用 PyTorch 1.6+）或 NVIDIA apex（‘apex’）。

In [ ]:
# 使用 PyTorch 内置 AMP，Trainer 默认使用
trainer = Trainer(amp_backend="native")

# 使用 NVIDIA Apex
trainer = Trainer(amp_backend="apex")

## amp_level

用于 16 位 GPU 精度的优化级别（O1、O2 等）（在幕后使用 NVIDIA apex）。

检查 NVIDIA apex 文档以了解级别

例子：

In [ ]:
# default used by the Trainer
trainer = Trainer(amp_level='O2')

调用 tune 来查找批量大小## auto_scale_batch_size

在任何训练之前，自动尝试找到适合内存的最大批量大小。

In [ ]:
# Trainer 使用的默认值（不缩放批量大小）
trainer = Trainer(auto_scale_batch_size=None)

# 运行批量缩放，结果覆盖 hparams.batch_size
trainer = Trainer(auto_scale_batch_size="binsearch")

# 调用 tune 来查找批量大小
trainer.tune(model)

## auto_select_gpus

如果启用并且 gpus 是一个整数，则自动选择可用的 gpus。 当 GPU 被配置为“独占模式”时，这尤其有用，这样一次只有一个进程可以访问它们。

例子：

In [ ]:
# 没有自动选择（选择系统上的前 2 个 gpu，如果其他进程正在占用可能会失败）
trainer = Trainer(gpus=2, auto_select_gpus=False)

# 启用自动选择（将在系统上找到两个可用的 GPU）
trainer = Trainer(gpus=2, auto_select_gpus=True)

# 指定所有 GPU，无论其可用性如何
Trainer(gpus=-1, auto_select_gpus=False)

# 指定所有可用的GPU（如果只有一个GPU未被占用，则使用一个gpu）
Trainer(gpus=-1, auto_select_gpus=True)

## auto_lr_find

在调用 trainer.tune() 时运行学习率查找算法（参见本文），以找到最佳初始学习率。

In [ ]:
# 培训师使用的默认值（无学习率查找器）
trainer = Trainer(auto_lr_find=False)

例子：

In [ ]:
# 运行学习率查找器，结果覆盖 hparams.learning_rate
trainer = Trainer(auto_lr_find=True)

# 调用 tune 查找 lr
trainer.tune(model)

例子：

In [ ]:
# 运行学习率查找器，结果覆盖 hparams.my_lr_arg
trainer = Trainer(auto_lr_find='my_lr_arg')

# call tune to find the lr
trainer.tune(model)

## benchmark

如果 true 启用 `cudnn.benchmark`。 如果您的输入大小不变，此标志可能会提高您的系统速度。 但是，如果确实如此，则可能会使您的系统变慢。

加速来自允许 cudnn 自动调谐器为硬件找到最佳算法 [请参阅此处的讨论]。

例子：

In [ ]:
# default used by the Trainer
trainer = Trainer(benchmark=False)

## deterministic

如果 true 启用 cudnn.deterministic。 可能会使您的系统变慢，但可确保重现性。 还设置 $HOROVOD_FUSION_THRESHOLD=0。

有关更多信息，请查看 [pytorch 文档]。

例子：

In [ ]:
# default used by the Trainer
trainer = Trainer(deterministic=False)

## callbacks

添加回调列表。 回调按此处定义的顺序依次运行，但 ModelCheckpoint 回调除外，它在所有其他回调之后运行以确保所有状态都保存到检查点。

In [ ]:
# a list of callbacks
callbacks = [PrintCallback()]
trainer = Trainer(callbacks=callbacks)

例子：

In [ ]:
from pytorch_lightning.callbacks import Callback

class PrintCallback(Callback):
    def on_train_start(self, trainer, pl_module):
        print("Training is started!")
    def on_train_end(self, trainer, pl_module):
        print("Training is done.")

也可以通过 configure_callbacks() 在 LightningModule 中添加特定于模型的回调。 此钩子中返回的回调将扩展最初提供给 Trainer 参数的列表，并在有两个或多个相同类型的情况下替换训练器回调。 ModelCheckpoint 回调总是最后运行。

## check_val_every_n_epoch

每 n 个训练周期检查一次 val。

例子：

In [ ]:
# default used by the Trainer
trainer = Trainer(check_val_every_n_epoch=1)

# 每 10 个训练周期运行一次 val 循环
trainer = Trainer(check_val_every_n_epoch=10)

## checkpoint_callback

默认情况下，Lightning 会在您当前的工作目录中为您保存一个检查点，以及您上次训练时期的状态，检查点会捕获模型使用的所有参数的确切值。 要禁用自动检查点，请将其设置为 False。

In [ ]:
# default used by Trainer
trainer = Trainer(checkpoint_callback=True)

# 关闭自动检查点
trainer = Trainer(checkpoint_callback=False)

您可以通过初始化 ModelCheckpoint 回调并将其添加到回调列表来覆盖默认行为。 有关如何自定义检查点，请参阅保存和加载权重。

In [ ]:
from pytorch_lightning.callbacks import ModelCheckpoint

# 初始化 ModelCheckpoint 回调，监控 'val_loss'
checkpoint_callback = ModelCheckpoint(monitor="val_loss")

# 将您的回调添加到回调列表中
trainer = Trainer(callbacks=[checkpoint_callback])

> 从 v1.1 开始不推荐将 ModelCheckpoint 实例传递给此参数，并且从 v1.3 开始不支持。 改用回调参数。

## default_root_dir

当没有记录器或 pytorch_lightning.callbacks.ModelCheckpoint 回调通过时，日志和权重的默认路径。 在某些集群上，您可能希望将日志和检查点的存储位置分开。 如果你不这样做，那么为了方便起见，请使用这个参数。 路径可以是本地路径或远程路径，例如 s3://bucket/path 或 ‘hdfs://path/’。 需要设置凭据才能使用远程文件路径。

In [ ]:
# default used by the Trainer
trainer = Trainer(default_root_dir=os.getcwd())

## fast_dev_run

如果设置为 n (int) 则运行 n 否则为 1 如果设置为 True batch(es) of train, val and test 以查找任何错误（即：一种单元测试）。

在使用单个批处理运行 fast_dev_run 时，伪代码如下所示：

In [ ]:
# loading
__init__()
prepare_data

# test training step
training_batch = next(train_dataloader)
training_step(training_batch)

# test val step
val_batch = next(val_dataloader)
out = validation_step(val_batch)
validation_epoch_end([out])

In [ ]:
# default used by the Trainer
trainer = Trainer(fast_dev_run=False)

# 运行 1 个训练、验证、测试批次和程序结束
trainer = Trainer(fast_dev_run=True)

# 运行 7 个训练、验证、测试批次和程序结束
trainer = Trainer(fast_dev_run=7)

> 这个参数与 limit_train/val/test_batches 有点不同。 设置此参数将禁用调谐器、检查点回调、提前停止回调、记录器和记录器回调，如 LearningRateLogger 并且仅运行 1 个时期。 这必须仅用于调试目的。 limit_train/val/test_batches 只限制批次的数量，不会禁用任何东西。

## flush_logs_every_n_steps

经常将日志写入磁盘。

In [ ]:
# default used by the Trainer
trainer = Trainer(flush_logs_every_n_steps=100)

## gpus

* 要训练的 GPU 数量 (int)
* 或在哪些 GPU 上进行训练（列表）
* 可以处理字符串

In [ ]:
# default used by the Trainer (ie: train on CPU)
trainer = Trainer(gpus=None)

# equivalent
trainer = Trainer(gpus=0)

例子：

In [ ]:
# int: 在 2 gpu 上训练
trainer = Trainer(gpus=2)

# list: 在 GPU 1、4 上训练（通过总线顺序）
trainer = Trainer(gpus=[1, 4])
trainer = Trainer(gpus='1, 4') # equivalent

# -1: 在所有 gpu 上训练
trainer = Trainer(gpus=-1)
trainer = Trainer(gpus='-1') # equivalent

# 结合 num_nodes 跨节点在多个 GPU 上进行训练，总共使用 8 个 GPU
trainer = Trainer(gpus=2, num_nodes=4)

# 仅在 GPU 1 和 4 上跨节点训练
trainer = Trainer(gpus=[1, 4], num_nodes=4)

## gradient_clip_val

渐变剪裁值
* 0 表示不剪辑。

In [ ]:
# default used by the Trainer
trainer = Trainer(gradient_clip_val=0.0)

## limit_train_batches

要检查多少训练数据集。 在调试或测试在 epoch 结束时发生的事情时很有用。

In [ ]:
# default used by the Trainer
trainer = Trainer(limit_train_batches=1.0)

例子：

In [ ]:
# default used by the Trainer
trainer = Trainer(limit_train_batches=1.0)

# 每个 epoch 只运行 25% 的训练集
trainer = Trainer(limit_train_batches=0.25)

# 每个 epoch 只运行 10 批训练集
trainer = Trainer(limit_train_batches=10)

## limit_test_batches

要检查多少测试数据集。

In [ ]:
# 培训师使用的默认值
trainer = Trainer(limit_test_batches=1.0)

# 每个 epoch 只运行 25% 的测试集
trainer = Trainer(limit_test_batches=0.25)

# 仅运行 10 个批次
trainer = Trainer(limit_test_batches=10)

在多个测试数据加载器的情况下，限制分别适用于每个数据加载器。

## limit_val_batches

要检查多少验证数据集。 在调试或测试在 epoch 结束时发生的事情时很有用。

In [ ]:
# default used by the Trainer
trainer = Trainer(limit_val_batches=1.0)

# 每个 epoch 只运行 25% 的验证集
trainer = Trainer(limit_val_batches=0.25)

# 仅运行 10 个批次
trainer = Trainer(limit_val_batches=10)

在多个验证数据加载器的情况下，限制分别适用于每个数据加载器。

## log_every_n_steps

添加记录行的频率（不写入磁盘）

In [ ]:
# default used by the Trainer
trainer = Trainer(log_every_n_steps=50)

## log_gpu_memory

选项：
* None
* ‘min_max’
* ‘all’'

In [ ]:
# default used by the Trainer
trainer = Trainer(log_gpu_memory=None)

# 记录所有 GPU（仅在主节点上）
trainer = Trainer(log_gpu_memory="all")

# 仅记录主节点上的最小和最大内存
trainer = Trainer(log_gpu_memory="min_max")

可能会降低性能，因为它使用 nvidia-smi 的输出。

## logger

用于实验跟踪的记录器（或可迭代的记录器集合）。 True 值使用如下所示的默认 TensorBoardLogger。 False 将禁用日志记录。

In [ ]:
from pytorch_lightning.loggers import TensorBoardLogger

# default logger used by trainer
logger = TensorBoardLogger(save_dir=os.getcwd(), version=1, name="lightning_logs")
Trainer(logger=logger)

## max_epochs

一旦达到这个时期数就停止训练

In [ ]:
# default used by the Trainer
trainer = Trainer(max_epochs=1000)

## min_epochs

至少要进行这么多 epoch 的强制训练

In [ ]:
# default used by the Trainer
trainer = Trainer(min_epochs=1)

## max_steps

在达到此步数后停止训练 如果 max_steps 或 max_epochs 已达到（最早），则训练将停止。

In [ ]:
# Default (disabled)
trainer = Trainer(max_steps=None)

# Stop after 100 steps
trainer = Trainer(max_steps=100)

## min_steps

强制训练至少这些步数。 Trainer 将训练模型至少 min_steps 或 min_epochs（最新）。

In [ ]:
# Default (disabled)
trainer = Trainer(min_steps=None)

# Run at least for 100 steps (disable min_epochs)
trainer = Trainer(min_steps=100, min_epochs=0)

## max_time

设置训练的最长时间。 训练将在中期中断。 对于可自定义的选项，请使用 Timer 回调。

In [ ]:
# Default (disabled)
trainer = Trainer(max_time=None)

# 训练 12 小时后或达到 10 个时期时停止（字符串）
trainer = Trainer(max_time="00:12:00:00", max_epochs=10)

# 1天5小时后停止（字典）
trainer = Trainer(max_time={"days": 1, "hours": 5})

如果 max_time 与 min_steps 或 min_epochs 一起使用，则 min_* 要求始终具有优先权。

## num_nodes

用于分布式训练的 GPU 节点数。

In [ ]:
# default used by the Trainer
trainer = Trainer(num_nodes=1)

# to train on 8 nodes
trainer = Trainer(num_nodes=8)

## num_processes

要训练的进程数。 使用accelerator="ddp"时自动设置为GPU数量。 使用 Accelerator="ddp_cpu" 在没有 GPU 的机器上模拟分布式训练时，设置为大于 1 的数字。 这对调试很有用，但不会提供任何加速，因为单进程 Torch 已经有效地利用了多个 CPU。

In [ ]:
# Simulate DDP for debugging on your GPU-less laptop
trainer = Trainer(accelerator="ddp_cpu", num_processes=2)

## num_sanity_val_steps

在开始训练例程之前，健全性检查运行 n 批 val。 这可以捕获验证中的任何错误，而无需等待第一次验证检查。 训练器默认使用 2 个步骤。 在此处关闭或修改它。

In [ ]:
# default used by the Trainer
trainer = Trainer(num_sanity_val_steps=2)

# turn it off
trainer = Trainer(num_sanity_val_steps=0)

# check all validation data
trainer = Trainer(num_sanity_val_steps=-1)

除非 num_sanity_val_steps=0，否则此选项将重置验证数据加载器。

## overfit_batches

使用训练集的这么多数据。 如果非零，将使用相同的训练集进行验证和测试。 如果训练数据加载器有 shuffle=True，Lightning 会自动禁用它。

用于快速调试或故意过度拟合。

In [ ]:
# default used by the Trainer
trainer = Trainer(overfit_batches=0.0)

# 仅使用 1% 的训练集（并将训练集用于 val 和 test）
trainer = Trainer(overfit_batches=0.01)

# 在 10 个相同批次上过拟合
trainer = Trainer(overfit_batches=10)

## plugins

插件允许您连接任意后端、精确库、集群等。例如：
* DDP
* TorchElastic
* Apex

要定义您自己的行为，请子类化相关类并将其传入。这是链接您自己的 ClusterEnvironment 的示例。

In [ ]:
from pytorch_lightning.plugins.environments import ClusterEnvironment


class MyCluster(ClusterEnvironment):
    def master_address(self):
        return your_master_address

    def master_port(self):
        return your_master_port

    def world_size(self):
        return the_world_size


trainer = Trainer(plugins=[MyCluster()], ...)

## prepare_data_per_node

如果 True 将为每个节点在 LOCAL_RANK=0 上调用 prepare_data() 。 如果 False 只会从 NODE_RANK=0 调用，LOCAL_RANK=0

In [ ]:
# default
Trainer(prepare_data_per_node=True)

# use only NODE_RANK=0, LOCAL_RANK=0
Trainer(prepare_data_per_node=False)

## precision

Lightning 支持双精度 (64)、全精度 (32) 或半精度 (16) 训练。

半精度或混合精度是结合使用 32 位和 16 位浮点来减少模型训练期间的内存占用。 这可以提高性能，在现代 GPU 上实现 3 倍的加速。

In [ ]:
# default used by the Trainer
trainer = Trainer(precision=32, gpus=1)

# 16-bit precision
trainer = Trainer(precision=16, gpus=1)

# 64-bit precision
trainer = Trainer(precision=64, gpus=1)

在 TPU 上运行时，将使用 torch.float16，但张量打印仍将显示 torch.float32。

CPU 不支持 16 位精度。

> 使用 PYTORCH 1.6+ 时，Lightning 使用原生 AMP 实现来支持 16 位精度。 NVIDIA APEX 库支持 PYTORCH < 1.6 的 16 位精度。

## process_position

命令进度条。 在同一节点上运行多个训练器时很有用。

In [ ]:
# default used by the Trainer
trainer = Trainer(process_position=0)

> 如果将自定义回调传递给回调，则忽略此参数。

## profiler

在训练期间分析各个步骤并协助确定瓶颈。

请参阅分析器文档。 更多细节。

In [ ]:
from pytorch_lightning.profiler import SimpleProfiler, AdvancedProfiler

# default used by the Trainer
trainer = Trainer(profiler=None)

# 分析标准训练事件，相当于`profiler=SimpleProfiler()`
trainer = Trainer(profiler="simple")

# 函数级统计信息的高级分析器，相当于`profiler=AdvancedProfiler()`
trainer = Trainer(profiler="advanced")

## progress_bar_refresh_rate

刷新进度条的频率（按步骤）。

In [ ]:
# default used by the Trainer
trainer = Trainer(progress_bar_refresh_rate=1)

# 禁用进度条
trainer = Trainer(progress_bar_refresh_rate=0)

> * 在 Google Colab 笔记本中，已知更快的刷新率（较低的数字）会导致它们崩溃，因为它们的屏幕刷新率。 如果用户不提供值，Lightning 会在这些环境中将其设置为 20。
> * 如果将自定义回调传递给回调，则忽略此参数。

## reload_dataloaders_every_n_epochs

设置为正整数以每 n 个时期重新加载数据加载器。

In [ ]:
# if 0 (default)
train_loader = model.train_dataloader()
for epoch in epochs:
    for batch in train_loader:
        ...

# if a positive integer
for epoch in epochs:
    if not epoch % reload_dataloaders_every_n_epochs:
        train_loader = model.train_dataloader()
    for batch in train_loader:
        ...

## replace_sampler_ddp

启用 DistributedSampler 的自动添加。 在 PyTorch 中，您必须在分布式设置中使用它，例如 TPU 或多节点。 采样器确保每个 GPU 都能看到数据的适当部分。 默认情况下，它会为训练采样器添加 shuffle=True，为 val/test 采样器添加 shuffle=False。 如果你想自定义它，你可以设置replace_sampler_ddp=False并添加你自己的分布式采样器。 如果 replace_sampler_ddp=True 并且已经添加了分布式采样器，Lightning 不会替换现有的采样器。

In [ ]:
# default used by the Trainer
trainer = Trainer(replace_sampler_ddp=True)

通过设置为 False，您必须添加自己的分布式采样器：

In [ ]:
# in your LightningModule or LightningDataModule
def train_dataloader(self):
    # default used by the Trainer
    sampler = torch.utils.data.distributed.DistributedSampler(dataset, shuffle=True)
    dataloader = DataLoader(dataset, batch_size=32, sampler=sampler)
    return dataloader

> 对于可迭代数据集，我们不会自动执行此操作。

## resume_from_checkpoint

要从此处路径中的特定检查点通过恢复训练。 如果从中期检查点恢复，训练将从下一个 epoch 开始。

In [ ]:
# default used by the Trainer
trainer = Trainer(resume_from_checkpoint=None)

# 从特定检查点恢复
trainer = Trainer(resume_from_checkpoint="some/path/to/my_checkpoint.ckpt")

## sync_batchnorm

在所有 GPU 上启用批处理归一化层之间的同步。

In [ ]:
trainer = Trainer(sync_batchnorm=True)

## track_grad_norm

* 无跟踪 (-1)
* 否则跟踪该范数（2 代表 2 范数）

In [ ]:
# default used by the Trainer
trainer = Trainer(track_grad_norm=-1)

# track the 2-norm
trainer = Trainer(track_grad_norm=2)

## tpu_cores

* 要在多少个 TPU 内核上进行训练（1 或 8 个）。
* 在哪个 TPU 核心上训练 [1-8]

单个 TPU v2 或 v3 具有 8 个内核。 一个 TPU pod 最多有 2048 个内核。 一片 POD 意味着您可以获得所需数量的内核。

您的有效批量大小为 batch_size * tpu 核心总数。

此参数可以是 1 或 8。

例子：

In [ ]:
# your_trainer_file.py

# default used by the Trainer (ie: train on CPU)
trainer = Trainer(tpu_cores=None)

# int: 在单核上训练
trainer = Trainer(tpu_cores=1)

# list: 在单个选定的核心上训练
trainer = Trainer(tpu_cores=[2])

# int: 在某些时候的核心上训练
trainer = Trainer(tpu_cores=8)

# 对于 8 个以上的内核，必须通过 xla 脚本提交，并指定最多 8 个内核。 
# XLA 脚本会将脚本复制到 POD 中的每个 TPU 上
trainer = Trainer(tpu_cores=8)

要在超过 8 个内核（即：一个 POD）上进行训练，请使用 xla_dist 脚本提交此脚本。

例子：

In [ ]:
python -m torch_xla.distributed.xla_dist
--tpu=$TPU_POD_NAME
--conda-env=torch-xla-nightly
--env=XLA_USE_BF16=1
-- python your_trainer_file.py

## val_check_interval

在一个训练时期内检查验证集的频率。 可以指定为 float 或 int。
* 使用 (float) 在训练时期内检查
* 使用 (int) 检查每 n 步（批次）

In [ ]:
# default used by the Trainer
trainer = Trainer(val_check_interval=1.0)

# 在训练时期检查验证集 4 次
trainer = Trainer(val_check_interval=0.25)

# 当使用 iterableDataset 并且您的数据集没有长度（即：具有流数据的生产案例）时，
# 每 1000 个训练批次检查验证集使用它
trainer = Trainer(val_check_interval=1000)

In [ ]:
# 这是估计一个时期内看到的批次总数的计算。

# 查找训练批次的总数
total_train_batches = total_train_samples // (train_batch_size * world_size)

# 计算在训练循环期间我们将调用验证的次数
val_check_batch = max(1, int(total_train_batches * val_check_interval))
val_checks_per_epoch = total_train_batches / val_check_batch

# 查找验证批次的总数
total_val_batches = total_val_samples // (val_batch_size * world_size)

# 运行的批次总数
total_fit_batches = total_train_batches + total_val_batches

## weights_save_path

指定保存权重的位置的目录。

In [ ]:
# default used by the Trainer
trainer = Trainer(weights_save_path=os.getcwd())

# save to your custom path
trainer = Trainer(weights_save_path="my/path")

例子：

In [ ]:
# 如果使用检查点回调，则覆盖权重路径
# **注意：这将权重保存到 some/path 而不是 my/path
checkpoint = ModelCheckpoint(dirpath='some/path')
trainer = Trainer(
    callbacks=[checkpoint],
    weights_save_path='my/path'
)

## weights_summary

训练开始时打印权重摘要。 选项：‘full’, ‘top’, None.。

In [ ]:
# 培训师使用的默认值（即：打印顶级模块的摘要）
trainer = Trainer(weights_summary="top")

# 打印所有模块和子模块的完整摘要
trainer = Trainer(weights_summary="full")

# 不要打印摘要
trainer = Trainer(weights_summary=None)

# Trainer类API

## 方法

### init

### fit

### validate

### test

### predict

### tune

## 属性

### callback_metrics

可用于回调的指标。 这些是在您通过 self.log 登录时自动设置的

In [ ]:
def training_step(self, batch, batch_idx):
    self.log("a_val", 2)


callback_metrics = trainer.callback_metrics
assert callback_metrics["a_val"] == 2

### current_epoch

现在的epoch

In [ ]:
def training_step(self, batch, batch_idx):
    current_epoch = self.trainer.current_epoch
    if current_epoch > 100:
        # do something
        pass

### logger (p)

当前正在使用的记录器。 这是一个使用tensorboard的例子

In [ ]:
def training_step(self, batch, batch_idx):
    logger = self.trainer.logger
    tensorboard = logger.experiment

### logged_metrics

发送到记录器（可视化器）的指标。

In [ ]:
def training_step(self, batch, batch_idx):
    self.log("a_val", 2, log=True)


logged_metrics = trainer.logged_metrics
assert logged_metrics["a_val"] == 2

### log_dir

当前实验的目录。 使用它来保存图像，等等...

In [ ]:
def training_step(self, batch, batch_idx):
    img = ...
    save_img(img, self.trainer.log_dir)

### is_global_zero

这个过程是否是多节点训练中的全局零

In [ ]:
def training_step(self, batch, batch_idx):
    if self.trainer.is_global_zero:
        print("in node 0, accelerator 0")

### progress_bar_metrics

发送到进度条的指标。

In [ ]:
def training_step(self, batch, batch_idx):
    self.log("a_val", 2, prog_bar=True)


progress_bar_metrics = trainer.progress_bar_metrics
assert progress_bar_metrics["a_val"] == 2